In [1]:
import csv
import random
import torch

from reasoner import Reasoner
from world import World

from collections import defaultdict
from tqdm import tqdm

In [2]:
random.seed(42)
torch.manual_seed(42)

model_lr = {
    'rl-property': 2.5074850299401197e-06,
    'bl-property': 2.508483033932136e-06,
    'axxl-property': 3.0054890219560877e-06,
}

negative_sampler = {
    'axxl-property': '../../induction/checkpoints/finetuned_models/axxl-property',
    'bl-property': '../../induction/checkpoints/finetuned_models/bl-property',
    'rl-property': '../../induction/checkpoints/finetuned_models/rl-property',
}

world = World(concept_path = "../data/concept_senses.csv", 
             feature_path = '../data/experimental splits/train_1ns.csv', 
             matrix_path = "../data/train_1ns_matrix.txt")
world.create()

521it [00:00, 6818.41it/s]


In [3]:
PROPERTIES = ['can dax', 'can fep', 'is vorpal', 'is mimsy', 'has blickets', 'has feps', 'is a wug', 'is a tove']

In [4]:
MODELS = ['axxl-property', 'bl-property', 'rl-property']
DEVICE = 'cuda:0'

In [5]:
categories = ['bird.n.01', 'mammal.n.01', 'reptile.n.01', 'fish.n.01', 'insect.n.01', 'mollusk.n.01']

candidates = []
for cat in categories:
    descendants = world.taxonomy[cat].descendants()
    for concept in descendants:
        negatives = set(categories) - set([cat])
        positive = [c for c in world.taxonomy[cat].descendants() if c != concept]
        for cn in negatives:
            negative = [c for c in world.taxonomy[cn].descendants()]

            sim_p = world.similarity([concept], positive).mean().item()
            sim_n = world.similarity([concept], negative).mean().item()

            if sim_n > sim_p:
                candidates.append((concept, cat, cn))

In [6]:
candidates

[('hippo', 'mammal.n.01', 'reptile.n.01'),
 ('dolphin', 'mammal.n.01', 'fish.n.01'),
 ('whale', 'mammal.n.01', 'fish.n.01'),
 ('turtle', 'reptile.n.01', 'fish.n.01'),
 ('slug', 'mollusk.n.01', 'insect.n.01'),
 ('snail', 'mollusk.n.01', 'insect.n.01')]

In [7]:
random.seed(42)

stimuli = []
for concept, p, n in candidates:
    negative = world.taxonomy[n].descendants()
    positive = [c for c in world.taxonomy[p].descendants() if c != concept]
#     positive = random.sample(positive, min(len(positive), len(negative)))
    
    stimuli.append([concept, positive, negative])

In [8]:
concept = 'oyster'
negative = 'mollusk'

world.similarity([concept], [c for c in world.taxonomy[f'{negative}.n.01'].descendants() if c != concept]).mean()

tensor(0.4069)

In [9]:
# precompute random samples

fish = [c for c in world.taxonomy['fish.n.01'].descendants()]

neg_samples = {
    'dolphin': random.sample([c for c in world.taxonomy['mammal.n.01'].descendants() if c != 'dolphin'], len(fish)),
    'whale': random.sample([c for c in world.taxonomy['mammal.n.01'].descendants() if c != 'whale'], len(fish))
}

In [10]:
results = defaultdict(dict)

for MODEL in MODELS:
    result = {'within': [], 'outside': []}
    for prop in tqdm(PROPERTIES):
        
        for stimulus in stimuli:
            reasoner = Reasoner(f'../../induction/checkpoints/finetuned_models/{MODEL}',
                    learning_rate = model_lr[MODEL], 
                    lexicon = world.lexicon,
                    device = DEVICE)
            
            concept, within, outside = stimulus
            
            within_stimuli = reasoner.prepare_stimuli(within, prop)
            outside_stimuli = reasoner.prepare_stimuli(outside, prop)
            adaptation = reasoner.prepare_stimuli([concept], prop)
            
            reasoner.adapt(adaptation, labels = torch.tensor([1]))
            
            generalization_within = reasoner.generalize(within_stimuli)[:,1].mean().item()
            generalization_outside = reasoner.generalize(outside_stimuli)[:,1].mean().item()
            
            print('================')
            print(f'Property: {prop}, Concept: {concept}')
            print(f'WITHIN: {generalization_within}')
            print(f'OUTSIDE: {generalization_outside}')
            print('================')
            
            result['within'].append(generalization_within)
            result['outside'].append(generalization_outside)
            
    results[MODEL] = result
            

  0%|                                                     | 0/8 [00:00<?, ?it/s]

Property: can dax, Concept: hippo
WITHIN: -0.07642970234155655
OUTSIDE: -0.0909963846206665
Property: can dax, Concept: dolphin
WITHIN: -0.244098961353302
OUTSIDE: -1.0315020084381104
Property: can dax, Concept: whale
WITHIN: -0.07020338624715805
OUTSIDE: -0.4079572260379791
Property: can dax, Concept: turtle
WITHIN: -0.05505573749542236
OUTSIDE: -0.5107094645500183
Property: can dax, Concept: slug
WITHIN: -0.6792489290237427
OUTSIDE: -0.12208029627799988


 12%|█████▋                                       | 1/8 [00:10<01:11, 10.16s/it]

Property: can dax, Concept: snail
WITHIN: -0.6891070604324341
OUTSIDE: -0.09203670918941498
Property: can fep, Concept: hippo
WITHIN: -0.3531833589076996
OUTSIDE: -0.3986625373363495
Property: can fep, Concept: dolphin
WITHIN: -0.42197185754776
OUTSIDE: -1.0740309953689575
Property: can fep, Concept: whale
WITHIN: -0.10421770066022873
OUTSIDE: -0.43521150946617126
Property: can fep, Concept: turtle
WITHIN: -0.2164015769958496
OUTSIDE: -0.593176007270813
Property: can fep, Concept: slug
WITHIN: -0.6576176285743713
OUTSIDE: -0.4647241234779358


 25%|███████████▎                                 | 2/8 [00:18<00:53,  9.00s/it]

...

Property: has feps, Concept: snail
WITHIN: -0.4503951072692871
OUTSIDE: -0.4711483418941498
Property: is a wug, Concept: hippo
WITHIN: -0.623667299747467
OUTSIDE: -0.5807585120201111
Property: is a wug, Concept: dolphin
WITHIN: -0.19166520237922668
OUTSIDE: -0.4693869650363922
Property: is a wug, Concept: whale
WITHIN: -0.18393288552761078
OUTSIDE: -0.4394706189632416
Property: is a wug, Concept: turtle
WITHIN: -0.12631168961524963
OUTSIDE: -0.4708830714225769
Property: is a wug, Concept: slug
WITHIN: -1.6881883144378662
OUTSIDE: -1.8395434617996216


 88%|███████████████████████████████████████▍     | 7/8 [01:19<00:10, 10.59s/it]

Property: is a wug, Concept: snail
WITHIN: -0.36286428570747375
OUTSIDE: -0.46361976861953735
Property: is a tove, Concept: hippo
WITHIN: -0.411854088306427
OUTSIDE: -0.3129456341266632
Property: is a tove, Concept: dolphin
WITHIN: -0.248368039727211
OUTSIDE: -0.49957016110420227
Property: is a tove, Concept: whale
WITHIN: -0.2971685826778412
OUTSIDE: -0.4817489981651306
Property: is a tove, Concept: turtle
WITHIN: -0.5342074036598206
OUTSIDE: -1.1717466115951538
Property: is a tove, Concept: slug
WITHIN: -0.5342559814453125
OUTSIDE: -0.5726107954978943


100%|█████████████████████████████████████████████| 8/8 [01:30<00:00, 11.33s/it]

Property: is a tove, Concept: snail
WITHIN: -0.3549792468547821
OUTSIDE: -0.40134039521217346


In [11]:
for k, v in results.items():
    print(f"MODEL: {k}")
    print(f"WITHIN: {torch.tensor(v['within']).mean()} (+= {torch.tensor(v['within']).std()})")
    print(f"OUTSIDE: {torch.tensor(v['outside']).mean()} (+= {torch.tensor(v['outside']).std()})")

MODEL: axxl-property
WITHIN: -0.45044490694999695 (+= 0.23141267895698547)
OUTSIDE: -0.548091471195221 (+= 0.2752586603164673)
MODEL: bl-property
WITHIN: -0.36042001843452454 (+= 0.22106441855430603)
OUTSIDE: -0.5001537203788757 (+= 0.22716650366783142)
MODEL: rl-property
WITHIN: -0.360299676656723 (+= 0.2796356678009033)
OUTSIDE: -0.5192589163780212 (+= 0.3136405348777771)


In [12]:
with open("../data/results/tease_apart.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(['model', 'within', 'outside'])
    for k,v in results.items():
        for w, o in zip(v['within'], v['outside']):
            writer.writerow([k, w, o])
#     print("=" * 80)
#     print(f"a <- c({', '.join([str(x) for x in v['within']])})")
#     print(f"b <- c({', '.join([str(x) for x in v['outside']])})")
#     print('t.test(a, b, paired = TRUE)')

In [5]:
PROPERTY = 'has feps'
CONCEPT = 'dolphin'

adaptation = reasoner.prepare_stimuli([CONCEPT], PROPERTY)
reasoner.adapt(adaptation, labels = torch.tensor([1]))


# birds = [c for c in world.taxonomy['bird.n.01'].descendants()]
mammals = [c for c in world.taxonomy['mammal.n.01'].descendants() if c != 'dolphin']
fish = [c for c in world.taxonomy['fish.n.01'].descendants() if c != 'dolphin']

mammal_stimuli = reasoner.prepare_stimuli(mammals, PROPERTY)
fish_stimuli = reasoner.prepare_stimuli(fish, PROPERTY)
# bird_stimuli = reasoner.prepare_stimuli(birds, PROPERTY)

In [6]:
reasoner.generalize(mammal_stimuli)[:,1].mean()

tensor(-0.2595)

In [7]:
reasoner.generalize(fish_stimuli)[:,1].mean()

tensor(-0.5563)

In [5]:
PROPERTY = 'has feps'

adaptation = reasoner.prepare_stimuli(["dolphin"], PROPERTY)
reasoner.adapt(adaptation, labels = torch.tensor([1]))


mammals = [c for c in world.taxonomy['mammal.n.01'].descendants() if c != 'dolphin']
fish = [c for c in world.taxonomy['fish.n.01'].descendants() if c != 'dolphin']

mammal_stimuli = reasoner.prepare_stimuli(mammals, PROPERTY)
fish_stimuli = reasoner.prepare_stimuli(fish, PROPERTY)

In [6]:
reasoner.generalize(mammal_stimuli)[:, 1].mean()

tensor(-0.1707)

In [7]:
reasoner.generalize(fish_stimuli)[:, 1].mean()

tensor(-0.6013)

In [15]:
world.similarity(['dolphin'], mammals).mean().item()

0.27070605754852295

In [16]:
world.similarity(['dolphin'], fish).mean().item()

0.30332738161087036

In [5]:
def prepare_stimuli_prop(self, concepts, properties):
    sentences = [f'{c} {p}.' for c in concepts for p in properties]
    print(sentences)
    encoded = self.tokenizer(sentences, return_tensors = 'pt', padding = True)
    
    return encoded

In [10]:
properties = ['is an animal', 'is a mammal', 'is an ape', 'can breathe', 'can grow', 'is a living thing', 'is a plant', 'is a fish']

stimuli = prepare_stimuli_prop(reasoner, ['a bird'], properties)

['a bird is an animal.', 'a bird is a mammal.', 'a bird is an ape.', 'a bird can breathe.', 'a bird can grow.', 'a bird is a living thing.', 'a bird is a plant.', 'a bird is a fish.']


In [11]:
reasoner.generalize(stimuli)[:,1].exp()

tensor([0.9936, 0.0117, 0.0059, 0.9947, 0.9946, 0.9874, 0.0046, 0.0063])

In [7]:
PROPERTY = "is a bird"
adaptation = prepare_stimuli_prop(reasoner, ["a dax"], [PROPERTY])
reasoner.adapt(adaptation, labels = torch.tensor([1]))

['a dax is a bird.']


In [8]:
properties = ['is an animal', 'is a mammal', 'is a bird', 'is an ape', 'can breathe', 'can grow', 'is a living thing', 'is a plant', 'is a fish', 'can fly']

stimuli = prepare_stimuli_prop(reasoner, ['a dax'], properties)

['a dax is an animal.', 'a dax is a mammal.', 'a dax is a bird.', 'a dax is an ape.', 'a dax can breathe.', 'a dax can grow.', 'a dax is a living thing.', 'a dax is a plant.', 'a dax is a fish.', 'a dax can fly.']


In [9]:
reasoner.generalize(stimuli)

tensor([[-4.4052, -0.0123],
        [-2.5386, -0.0823],
        [-3.4440, -0.0325],
        [-3.1044, -0.0459],
        [-4.6169, -0.0099],
        [-4.8222, -0.0081],
        [-3.4630, -0.0318],
        [-1.9329, -0.1563],
        [-1.0055, -0.4555],
        [-3.3311, -0.0364]])